<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Feature_Importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libaries

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

## Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/contract_classification.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-0b00313732a5>:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


     ANO_SID CORPORATE_DEVISION   ORTPLZ  ORTS-NAME               STRASSE  \
0  4114028.0                VHV  42109.0  Wuppertal     Hans-Böckler-Str.   
1  4114039.0                VHV  42277.0  Wuppertal       Liegnitzer Str.   
2  4114045.0                VHV  42389.0  Wuppertal             Rascheweg   
3  4114049.0                VGV  42277.0  Wuppertal               Am Diek   
4  4114055.0                VHV  42553.0    Velbert  Emil-Schniewind-Str.   

   SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR        WFL ZONE  ...  \
0      71081.0      NORMAL_VENTURE        1967.565648   69.00000  2.0  ...   
1      55708.0      NORMAL_VENTURE        1967.565648   65.00000  4.0  ...   
2      74148.0      DESIGN_CLASS_I        1967.565648   75.00000  1.0  ...   
3     664000.0      NORMAL_VENTURE        1967.565648  106.24368  NaN  ...   
4      75682.0      NORMAL_VENTURE        1967.565648  119.00000  2.0  ...   

   DRAIN_PIPE_INSURED  PRODUCTLINE  PRIOR_DAMAGES UVV-KZ  UNDERWRITE

In [3]:
print(contract_class.info)

<bound method DataFrame.info of               ANO_SID CORPORATE_DEVISION   ORTPLZ  ORTS-NAME  \
0           4114028.0                VHV  42109.0  Wuppertal   
1           4114039.0                VHV  42277.0  Wuppertal   
2           4114045.0                VHV  42389.0  Wuppertal   
3           4114049.0                VGV  42277.0  Wuppertal   
4           4114055.0                VHV  42553.0    Velbert   
...               ...                ...      ...        ...   
11574434  145240866.0                VGV  41844.0    Wegberg   
11574435  145240867.0                VHV   4758.0    Oschatz   
11574436  145240892.0                VHV  26316.0      Varel   
11574437  145240908.0                VHV  53332.0   Bornheim   
11574438  145240911.0                VHV  46397.0    Bocholt   

                       STRASSE  SUM_INSURED CONSTRACTION_DESIGN  \
0            Hans-Böckler-Str.      71081.0      NORMAL_VENTURE   
1              Liegnitzer Str.      55708.0      NORMAL_VENTURE  

In [4]:
print(contract_class['DAMAGE'].value_counts())

DAMAGE
0    11389665
1      184774
Name: count, dtype: int64


## Search for Important features

# Preprocess the data

In [28]:
#Choose variables where it makes logical sense
#For now, leave out Ort, becuase it takes too long to encode to binary

initial_features = contract_class.loc[:, ['CORPORATE_DEVISION',  'SUM_INSURED', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED','PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE']]

initial_features_clean = initial_features.dropna()

print(initial_features.info())

print(initial_features_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11574439 entries, 0 to 11574438
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   CORPORATE_DEVISION   object 
 1   SUM_INSURED          float64
 2   CONSTRACTION_DESIGN  object 
 3   CONSTRUCTION_YEAR    float64
 4   WFL                  float64
 5   ZONE                 object 
 6   DRAIN_PIPE_INSURED   int64  
 7   PRODUCTLINE          object 
 8   PRIOR_DAMAGES        int64  
 9   UVV-KZ               int64  
 10  UNDERWRITER          object 
 11  DAMAGE               int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 1.0+ GB
None
<class 'pandas.core.frame.DataFrame'>
Index: 8718680 entries, 0 to 11574438
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   CORPORATE_DEVISION   object 
 1   SUM_INSURED          float64
 2   CONSTRACTION_DESIGN  object 
 3   CONSTRUCTION_YEAR    float64
 4   WFL                  flo

In [29]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.head())

   SUM_INSURED  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  PRIOR_DAMAGES  \
0      71081.0        1967.565648   69.0                   0              0   
1      55708.0        1967.565648   65.0                   0              0   
2      74148.0        1967.565648   75.0                   0              0   
4      75682.0        1967.565648  119.0                   0              0   
5      70000.0        1967.565648  100.0                   0              0   

   UVV-KZ  DAMAGE  CORPORATE_DEVISION_VHV  \
0       1       0                    True   
1       1       0                    True   
2       1       0                    True   
4       1       0                    True   
5       1       0                    True   

   CONSTRACTION_DESIGN_CARAVAN_MOTORHOME  CONSTRACTION_DESIGN_DESIGN_CLASS_I  \
0                                  False                               False   
1                                  False                               False   
2             

# Find feature Importance

In [30]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

# Split into test and training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
#initialize Random Forrest
rf_regressor = RandomForestRegressor(n_estimators=10, random_state=42)

# Train model
rf_regressor.fit(X_train, y_train)

# Feature-importnace analysis
feature_importance = rf_regressor.feature_importances_

# Display results
print("Feature Importance:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {feature_importance[i]}")

Feature Importance:
SUM_INSURED: 0.661095075202173
CONSTRUCTION_YEAR: 0.16712853767188202
WFL: 0.05114504662127624
DRAIN_PIPE_INSURED: 0.0
PRIOR_DAMAGES: 0.0063496037693985385
UVV-KZ: 0.0016713517086635594
CORPORATE_DEVISION_VHV: 0.007194369093985355
CONSTRACTION_DESIGN_CARAVAN_MOTORHOME: 1.9280094501918083e-08
CONSTRACTION_DESIGN_DESIGN_CLASS_I: 0.00015042762011887562
CONSTRACTION_DESIGN_DESIGN_CLASS_II: 3.1008090649461246e-08
CONSTRACTION_DESIGN_DESIGN_CLASS_III: 9.856311188090109e-07
CONSTRACTION_DESIGN_DESIGN_CLASS_IV: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_V: 1.5884843025679944e-08
CONSTRACTION_DESIGN_NORMAL_VENTURE: 0.0017898293019200015
CONSTRACTION_DESIGN_PREDOMINANTLY_WOODEN_ROOF: 4.220582809231791e-05
CONSTRACTION_DESIGN_PREFAB_HOUSE: 0.0015594915781360669
CONSTRACTION_DESIGN_PREFAB_HOUSE_I: 0.0
PRODUCTLINE_Kompakt: 0.01111716069676955
PRODUCTLINE_Plus: 0.006903883398996428
PRODUCTLINE_Premium: 0.005880452252091618
PRODUCTLINE_Sonst: 0.0013613094977586223
PRODUCTLINE_Top: 0.015